In [6]:
import xarray as xr
import numpy as np
import dask.array as da
import matplotlib.pyplot as plt
import os
from dask.distributed import Client, LocalCluster
from datetime import datetime,timedelta
import rioxarray
import xarray as xr
import glob
import rioxarray as rio
import cftime
import indices_function as ifun
from rechunker import rechunk


In [2]:
cluster = LocalCluster(
    n_workers=20, 
    threads_per_worker=1,
    timeout='3600s',
    memory_limit='40GB',   
)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 20
Total threads: 20,Total memory: 745.06 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35825,Workers: 20
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 745.06 GiB
Comm: tcp://127.0.0.1:35169,Total threads: 1
Dashboard: http://127.0.0.1:45841/status,Memory: 37.25 GiB
Nanny: tcp://127.0.0.1:42499,


In [ ]:
for year in range(1990,2015):
    file_names=glob("/nobackupp27/nex/projects/GDDP2/HI/BCSD/*/historical/*/tasmax/*_"+str(year)+".nc")

    # Open each file lazily using Dask and xarray
    datasets = [xr.open_dataset(file).chunk({"time":-1,"lat":400,"lon":400}) for file in file_names]
    def add_time(ds):
        ds_standard= ds.convert_calendar(calendar ='noleap', 
        align_on ='year', missing = np.nan)
        ds_standard['time'] = ds_standard['time'].dt.floor('D')
        return ds_standard
    datasets_with_time= [add_time(ds) for ds in datasets]

#     # Concatenate datasets along the time dimension using Dask and xarray
    combined_data = xr.concat(datasets_with_time, dim='files')

#     # Calculate the mean across all files using Dask and xarray (lazy computation)
    mean_data = combined_data.mean(dim='files')
    mean_data.to_zarr("/nobackupp28/skhajehe/hawaii_multi/tasmax/"+str(year)+".zarr",mode='w')



In [4]:
for year in range(1990,2015):
    file_names=glob("/nobackupp27/nex/projects/GDDP2/HI/BCSD/*/historical/*/tasmin/*_"+str(year)+".nc")

    # Open each file lazily using Dask and xarray
    datasets = [xr.open_dataset(file).chunk({"time":-1,"lat":400,"lon":400}) for file in file_names]
    def add_time(ds):
        ds_standard= ds.convert_calendar(calendar ='noleap', 
        align_on ='year', missing = np.nan)
        ds_standard['time'] = ds_standard['time'].dt.floor('D')
        return ds_standard
    datasets_with_time= [add_time(ds) for ds in datasets]

#     # Concatenate datasets along the time dimension using Dask and xarray
    combined_data = xr.concat(datasets_with_time, dim='files')

#     # Calculate the mean across all files using Dask and xarray (lazy computation)
    mean_data = combined_data.mean(dim='files')
    mean_data.to_zarr("/nobackupp28/skhajehe/hawaii_multi/tasmin/"+str(year)+".zarr",mode='w')



In [ ]:


# List of file names
models=glob.glob("/nobackupp27/nex/projects/GDDP2/HI/BCSD/*")
# model_tot=models[9:10]+models[10:20]+models[26:]
# model_tot.sort()
for model in models:
    # if model in done:
    #     print("done")
    #     continue
    try:
        file_names=glob.glob(model+"/historical/*/tasmax/*.nc")
        file_names.sort()
        # Open each file lazily using Dask and xarray
        datasets =xr.open_mfdataset(file_names[40:]).chunk({"time":-1,"lat":100,"lon":100})
        # Concatenate datasets along the time dimension using Dask and xarray
        quantile_models = ifun.nonzero_quantiles(datasets,[0.25,0.5,0.75,0.9])
        quantile_models.to_zarr("/nobackupp28/skhajehe/hawaii_multi/tasmax/quantiles/"+model.split("/")[-1]+".zarr",mode='w')
    except:
        print(model+" doesn't exist!")
        pass
        


2024-12-16 15:48:34,828 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-12-16 15:49:03,124 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-12-16 15:49:06,192 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-12-16 15:49:09,060 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
2024-12-16 15:49:12,070 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
2024-12-16 15:49:16,210 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


In [ ]:


# List of file names
models=glob.glob("/nobackupp27/nex/projects/GDDP2/HI/BCSD/*")
# model_tot=models[9:10]+models[10:20]+models[26:]
# model_tot.sort()
for model in models:
    # if model in done:
    #     print("done")
    #     continue
    try:
        file_names=glob.glob(model+"/historical/*/pr/*.nc")
        file_names.sort()
        # Open each file lazily using Dask and xarray
        datasets =xr.open_mfdataset(file_names[40:]).chunk({"time":-1,"lat":100,"lon":100})
        # Concatenate datasets along the time dimension using Dask and xarray
        quantile_models = ifun.nonzero_quantiles(datasets,[0.25,0.5,0.75,0.9])
        quantile_models.to_zarr("/nobackupp28/skhajehe/hawaii_multi/pr/quantiles/"+model.split("/")[-1]+".zarr",mode='w')
    except:
        print(model+" doesn't exist!")
        pass
        
